In [4]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
import numpy as np
from PIL import Image
import argparse
import math


theano.config.floatX = 'float32'
DATA_LOC=r"C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting"
FTRAIN   = 'train2014'
FVALID   = 'val2014'
CAPTION  = 'dict_key_imgID_value_caps_train_and_valid.pkl'
noise_dim=10


def load(batch_idx = None, batch_size = None, valid = False):
    '''Load data from FVALID if *valid* is True, otherwise load from FTRAIN.
       Discard black and white examples by default.
    '''
    # Path to dataset
    fname = FVALID if valid else FTRAIN
    data_path = os.path.join(DATA_LOC, fname)
    print data_path + "/*.jpg"
    imgs = glob.glob(data_path + "/*.jpg")
    if batch_idx != None:
        batch_imgs = imgs[batch_idx*batch_size:(batch_idx+1)*batch_size]
    else:
        batch_imgs = imgs

    # Initialize variables
    X_int, y_int= [], []

    for i, img_path in enumerate(batch_imgs):
    # loading images from the given path
        img = Image.open(img_path)
        img_array = np.array(img)

        # Get input/target from the images
        center = (int(np.floor(img_array.shape[0] / 2.)), int(np.floor(img_array.shape[1] / 2.)))
        if len(img_array.shape) == 3:
            input = np.copy(img_array)
            input[center[0]-16:center[0]+16, center[1]-16:center[1]+16, :] = 0
            target = img_array[center[0]-16:center[0]+16, center[1] - 16:center[1]+16, :]
        
        X_int.append(input)
        y_int.append(target)

    X = np.asarray(X_int, dtype=np.float32) / 255.
    y = np.asarray(y_int, dtype=np.float32) / 255.
    X = np.transpose(X, (0,3,1,2))
    y = np.transpose(y, (0,3,1,2))

    return X, y



def rebuilt_image(X, ypd, display = True): 
    
    # Rearrange shapes, data type
    X = np.asarray(255*X.transpose(0,2,3,1), np.uint8)
    ypd = np.asarray(255*ypd.transpose(0,2,3,1), np.uint8)

    collage = []

    for i in range(len(X)):
        img_array = X[i,:]
        center = (int(np.floor(img_array.shape[0] / 2.)), int(np.floor(img_array.shape[1] / 2.)))
        # Draw predicted image
        imgpd = np.copy(img_array)
        imgpd[center[0]-16:center[0]+16, center[1]-16:center[1]+16, :] = ypd[i,:]
        if i == 0: collage = imgpd
        else:      collage = np.hstack((collage, imgpd))

    # Display images
    if display:
        Image.fromarray(collage).show()


def genrator_model():
    model=Sequential()
    
    model.add(Dense(input_shape=(Batch_size, noise_dim), units=3072, activation='relu'))
    model.add(Reshape([32, 32, 3]))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(5,5), padding='same', strides=(1,1), activation='relu')))
    model.add(BatchNormalization(Convolution2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu')))
    model.add(BatchNormalization(Convolution2D(filters=64, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu')))
    model.add(BatchNormalization(Convolution2D(filters=64, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu')))
    model.add(BatchNormalization(Convolution2D(filters=3, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu')))
    model.add(Reshape([32,32, 3]))
              
    
def discriminator_model():
    model=Sequential()
    model.add(BatchNormalization(Convolution2D(filters=32, input_shape=(32, 32, 3), kernel_size=(3,3), activation='relu', strides=(1,1), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=64, kernel_size=(3,3), activation='relu', strides=(2,2), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=64, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), activation='relu', strides=(2,2), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), activation='relu', strides=(2,2), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same')))
    model.add(BatchNormalization(Convolution2D(filters=32, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same')))
    model.add(Dense(1, activation='sigmoid'))
    return model
    
    
    
    
def generator_containing_discriminator():
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model
              
              
def Train(Batch_size):
    X_train, y_train=load(1,10)
    X_val, y_val=load(1, 10, valid=True)
    discriminator = discriminator_model()
    generator = generator_model()
    discriminator_on_generator = \
        generator_containing_discriminator(generator, discriminator)
    d_optim = SGD(lr=0.002,momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.002,momentum=0.9, nesterov=True)
    generator.compile(loss='mean_squared_error', optimizer='SGD')
    discriminator_on_generator.compile(
        loss='mean_squared_error', optimizer=g_optim)
    discriminator.trainable = True
    discriminator.compile(loss='mean_squared_error', optimizer=d_optim)
    img=load(1,1, valid=True)
    generated_image=generator.predict(img)
    noise = np.zeros((BATCH_SIZE, 10))
    for epoch in range(10):
              for index in range(int(X_train.shape[0]/BATCH_SIZE)):
                  for i in range(BATCH_SIZE):
                    noise[i, :] = np.random.uniform(-1, 1, 10)
                image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
                generated_images = generator.predict(noise, verbose=0)
            if index % 20 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save(
                    str(epoch)+"_"+str(index)+".png")
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = discriminator.train_on_batch(X, y)
            for i in range(BATCH_SIZE):
                noise[i, :] = np.random.uniform(-1, 1, 10)
            discriminator.trainable = False
            g_loss = discriminator_on_generator.train_on_batch(
                noise, [1] * BATCH_SIZE)
            discriminator.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if index % 10 == 9:
                generator.save_weights('generator', True)
                discriminator.save_weights('discriminator', True)  
              

    def generate(BATCH_SIZE, nice=False):
    generator = generator_model()
    generator.compile(loss='mean_squared_error', optimizer="SGD")
    generator.load_weights('generator')
    if nice:
        discriminator = discriminator_model()
        discriminator.compile(loss='mean_squared_error', optimizer="SGD") 
        discriminator.load_weights('discriminator')
        noise = np.zeros((BATCH_SIZE*20, 10))
        for i in range(BATCH_SIZE*20):
            noise[i, :] = np.random.uniform(-1, 1, 10)
        generated_images = generator.predict(noise, verbose=1)
        d_pret = discriminator.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE, 1) +
                               (generated_images.shape[2:]), dtype=np.float32)
        for i in range(int(BATCH_SIZE)):
            idx = int(pre_with_index[i][1])
            nice_images[i, 0, :, :] = generated_images[idx, 0, :, :]
        image = combine_images(nice_images)
    else:
        noise = np.zeros((BATCH_SIZE, 10))
        for i in range(BATCH_SIZE):
            noise[i, :] = np.random.uniform(-1, 1, 10)
        generated_images = generator.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*255+255
    Image.fromarray(image.astype(np.uint8)).save(
        "gen_image.png")

              
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", type=str)
    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--nice", dest="nice", action="store_true")
    parser.set_defaults(nice=False)
    args = parser.parse_args()
    return args
              

if __name__ == "__main__":
    args = get_args()
    if args.mode == "train":
        train(BATCH_SIZE=args.batch_size)
    elif args.mode == "generate":
        generate(BATCH_SIZE=args.batch_size, nice=args.nice)
              
              
    

IndentationError: unindent does not match any outer indentation level (<ipython-input-4-ee2b9edf873d>, line 147)